In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Encode text labels
label_encoder = LabelEncoder()

In [3]:
df_2018 = pd.read_parquet("./data/clean_joined_2018.parquet").dropna()

df_2018 = df_2018[:1_000_000]

In [4]:
df_2018["MFR"] = label_encoder.fit_transform(df_2018["MFR"])

X = df_2018.drop(columns=["Tail_Number", "N-NUMBER", "CODE", 'Cancelled', 'Diverted', 'DepDelayMinutes', 'ArrDelayMinutes'], axis=1)

y = {
	'Cancelled' : df_2018['Cancelled'].astype(bool), 
	'Diverted' : df_2018['Diverted'].astype(bool), 
	'DepDelayMinutes' : df_2018['DepDelayMinutes'].astype(float), 
	'ArrDelayMinutes' : df_2018['ArrDelayMinutes'].astype(float), 
}

In [5]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(
	max_depth=2, random_state=42
)

0.4135

In [6]:
for key in y.keys():
	current_y = y[key]
	
	X_train, X_test, y_train, y_test = train_test_split(
		X, 
		current_y, 
		test_size=0.3, 
		random_state=42
	)
	
	
	
	rfc.fit(X_train, y_train)
	
	rfc.score(X_test, y_test)

In [7]:


model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.5, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=29, ...)

In [8]:

from sklearn.model_selection import GridSearchCV

xgb_model = xgboost.XGBClassifier(
	n_jobs=-1,
	objective='multi:softprob',  # Essential for probability outputs [1][4][5]
	num_class=len(label_encoder.classes_),  # Automatically set class count
	tree_method='hist',  # Optimized for performance
	eval_metric='mlogloss',  # Recommended for multi-class [1][7]
	enable_categorical=False,  # Only needed if using categorical features
	n_estimators=100,
	device='cuda',
	learning_rate=0.5,
)

clf = GridSearchCV(
	xgb_model,
	{"max_depth": [2, 4, 6], "n_estimators": [50, 100, 200]},
	verbose=1,
	n_jobs=-1,
)

clf.fit(X_train, y_train)
print(clf.best_score_)
print(clf.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
0.41827500000000006
{'max_depth': 2, 'n_estimators': 50}


In [9]:

#params = {
#	'learning_rate': 0.25,
#	'max_depth': 6,
#	'subsample': 0.8,
#	'colsample_bytree': 0.7,
#	'tree_method': 'hist',
#	'device': 'cuda',
#	'num_class' : len(label_encoder.classes_),  # Automatically set class count
#}
#
#results = xgboost.cv(
#	params,
#	dtrain=xgboost.DMatrix(X_train, y_train),
#	num_boost_round=500,
#	early_stopping_rounds=20,
#	metrics=['merror', 'mlogloss'],
#	nfold=5,
#	verbose_eval=True
#	
#)